# Correcting songs and fine-tuing

## Import modules

In [22]:
import os
from neuspell import available_checkers, BertChecker, SclstmChecker

print(f"available checkers: {available_checkers()}")

available checkers: ['BertsclstmChecker', 'CnnlstmChecker', 'ElmosclstmChecker', 'NestedlstmChecker', 'SclstmChecker', 'SclstmbertChecker', 'SclstmelmoChecker', 'BertChecker']


## Define data path

In [23]:
clean_file = 'clean_en_lyrics.txt'
noisy_file = 'noisy_en_lyrics.txt'

train_data_path = '../dataset/train'
clean_train_path = os.path.join(train_data_path, clean_file)
noisy_train_path = os.path.join(train_data_path, noisy_file)

test_data_path = '../dataset/test'
clean_test_path = os.path.join(test_data_path, clean_file)
noisy_test_path = os.path.join(test_data_path, noisy_file)

## Load training data

In [24]:
with open(clean_train_path) as f:
    clean_train = f.read().splitlines()

clean_train[:10]

['well show me the way',
 "'cause nothin' from nothin' leaves nothin'",
 "she said look, what's your game baby",
 'savage love, did somebody, did somebody break your heart',
 "told you i'll be here forever",
 'oh baby, we found love right where we are maybe',
 'all i know is we said, "hello"',
 "another saturday night and i ain't got nobody",
 'if your schemes like your dreams',
 'you fell, i caught you']

In [25]:
with open(noisy_train_path) as f:
    noisy_train = f.read().splitlines()

noisy_train[:10]

['well show me the way',
 "'caue nothvin' from nothin' leaves nothin'",
 "sye said look, wha's yocur gmae baby",
 'svage live, did somebodvy, did somebody beak your heart',
 "tlod you i'll be helre forever",
 'oh baby, we found love riaght where we ahre mabe',
 'all i know is we saiid, "ello"',
 "another saturfay ngiht and i ain't got nobody",
 'if your schemes lkie your dreas',
 'you fell, i caught you']

In [26]:
with open(clean_test_path) as f:
    clean_test = f.read().splitlines()

clean_test[:10]

['\ufefftell her that she beautiful every day i remind her',
 'i get lost in her eyes like dust from the skies',
 "did i say that out loud i'm so crazy about mine",
 "but we don't have the same soul",
 'tell me that you love me baby say it again',
 "repeat it over and over until it's in my brain",
 "you need to send your location i can't think",
 'did i say that out loud',
 "i'm so crazy about mine",
 'when i look back']

In [27]:
with open(noisy_test_path) as f:
    noisy_test = f.read().splitlines()

noisy_test[:10]

['tell her that she beautiufl every dsy i remind her',
 'i get lost in her eyes licke dsut from tge skies',
 "did i say that out louud i'm so crazry about mnie",
 "but we don't have the saqme soul",
 'tell me that you love me babny say it again',
 "repeat it over and over until it's in my brian",
 "you need to send your location i can't think",
 'did i sqy that out luod',
 "i'm so craay about mibe",
 'when i look back']

## BERTChecker

### Load pretrained model

In [17]:
# create BertChecker
bert_checker = BertChecker()
# use pretrained model
bert_checker.from_pretrained()

loading vocab from path:e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise\vocab.pkl
initializing model
Number of parameters in the model: 185211810
Loading model params from checkpoint dir: e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise


### Evaluate pretrained model on testing set

In [18]:
bert_checker.evaluate(clean_file=clean_file, corrupt_file=noisy_file, data_dir=test_data_path)

../dataset/test clean_en_lyrics.txt noisy_en_lyrics.txt


930it [00:00, 464922.85it/s]
930it [00:00, 464756.67it/s]


loaded tuples of (corr,incorr) examples from ../dataset/test
###############################################
data size: 930


233it [00:59,  3.92it/s]



Epoch None valid_loss: 0.6766380007490738
total inference time for this data is: 59.443624 secs
###############################################


total token count: 6684
_corr2corr:4938, _corr2incorr:459, _incorr2corr:1085, _incorr2incorr:202
accuracy is 0.9011071214841412
word correction rate is 0.8430458430458431
precision is 0.914952751528627
recall is 0.9607003891050584
f1 score is 0.9372686722976179
###############################################


### Example of wrong outputs 

In [28]:
bert_correct = bert_checker.correct_strings(noisy_test)
bert_correct[:10]

['tell her that she beautiful every day I remind her',
 'I get lost in her eyes like dust from the skies',
 "did I say that out loud i ' m so crazy about mine",
 "but we don ' t have the same soul",
 'tell me that you love me baby say it again',
 "repeat it over and over until it ' s in my brain",
 "you need to send your location I can ' t think",
 'did I say that out loud',
 "I ' m so crazy about more",
 'when I look back']

In [29]:
clean_test_prep = [i.replace(' ', '').strip().lower() for i in clean_test]
bert_correct_prep = [i.replace(' ', '').strip().lower() for i in bert_correct]

print('len data:', len(clean_test_prep))
print('Number of correct data after bert:', sum(a == b for a, b in zip(bert_correct_prep, clean_test_prep)))

len data: 930
Number of correct data after bert: 641


In [30]:
for i in range(len(clean_test_prep)):
    if clean_test_prep[i] != bert_correct_prep[i]:
        print(clean_test[i], ' | ', bert_correct[i])

tell her that she beautiful every day i remind her  |  tell her that she beautiful every day I remind her
i'm so crazy about mine  |  I ' m so crazy about more
and if i do will you be there with me father sister brother  |  and if I do will you be there with my father sister brother
will i have one of mine  |  will I have one of five
i guess i'll be fine  |  I guess didn ' ll be fine
don't you dare say  |  dont you dare say
for me you are the one  |  for me you are the oak
baby don't you dare say  |  baby don ' t you dare baby
maybe i'll take my glasses off  |  maybe it ' ll take my glasses off
if don't really care see  |  if don ' t really care side
don't know what they mean they're special just for you  |  dont know what they mean they ' re special just for you
turn it up hot loving you is free  |  turn it up how looking you is free
that's what they all say  |  that ' s what the all sky
you believe me don't you baby  |  you believe me do ' nt you baby
i could fuck him in the range  |

### Fine-tuning BertChecker

In [12]:
# create new BertChecker
bert_checker_new = BertChecker()
# load pretrained model
bert_checker_new.from_pretrained()
# fine-tune on training data
bert_checker_new.finetune(clean_file=clean_file, corrupt_file=noisy_file, data_dir=train_data_path)

loading vocab from path:e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise\vocab.pkl
initializing model
Number of parameters in the model: 185211810
Loading model params from checkpoint dir: e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise


7309it [00:00, 664257.93it/s]
7309it [00:00, 1218206.55it/s]


loaded tuples of (corr,incorr) examples from ../dataset/train
len of train and test data:  5848 1461
CHECKPOINT_PATH: e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise\new_models\bert-base-cased-2
Training model params
In epoch: 0
train_data size: 5848
Percent: [>                             ] 1% || batch_time: 3.7478 || batch_loss: 0.3949 || avg_batch_loss: 0.3523 || batch_acc: 0.9381 || avg_batch_acc: 0.9381 

e:\NLP\nlp-env\lib\site-packages\pytorch_pretrained_bert\optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:882.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


Percent: [----------------------------->] 100% || batch_time: 2.9423 || batch_loss: 0.1038 || avg_batch_loss: 0.1967 || batch_acc: 0.9381 || avg_batch_acc: 0.9381 
Epoch 0 train_loss: 0.196699008597201
valid_data size: 1461
Percent: [----------------------------->] 100% || batch_time: 1.5124 || batch_loss: 0.0387 || avg_batch_loss: 0.1091 || batch_acc: 0.9944 || avg_batch_acc: 0.9761 
Epoch 0 valid_loss: 0.10908372323636127
validation accuracy improved from -1.0000 to 44.9012
Model saved at e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise\new_models\bert-base-cased-2\pytorch_model.bin in epoch 0
In epoch: 1
train_data size: 5848
Percent: [----------------------------->] 100% || batch_time: 3.5188 || batch_loss: 0.0074 || avg_batch_loss: 0.0167 || batch_acc: 0.9856 || avg_batch_acc: 0.9856 

...

Percent: [----------------------------->] 99% || batch_time: 3.3257 || batch_loss: 0.0018 || avg_batch_loss: 0.0070 || batch_acc: 1.0000 || avg_batch_acc: 1.0000 

Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.


Percent: [----------------------------->] 100% || batch_time: 2.7056 || batch_loss: 0.0008 || avg_batch_loss: 0.0069 || batch_acc: 1.0000 || avg_batch_acc: 1.0000 

Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.


Percent: [----------------------------->] 100% || batch_time: 2.6521 || batch_loss: 0.0002 || avg_batch_loss: 0.0069 || batch_acc: 1.0000 || avg_batch_acc: 1.0000 
Epoch 2 train_loss: 0.006928394167203567
valid_data size: 1461
Percent: [----------------------------->] 100% || batch_time: 1.6624 || batch_loss: 0.0353 || avg_batch_loss: 0.0971 || batch_acc: 0.9944 || avg_batch_acc: 0.9790 
Epoch 2 valid_loss: 0.09710691681982059
validation accuracy improved from 45.0326 to 45.0326
Model saved at e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise\new_models\bert-base-cased-2\pytorch_model.bin in epoch 2
Model and logs saved at e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise\new_models\bert-base-cased-2


### Evaluate on testing set after fine-tuning

In [32]:
bert_checker_new.evaluate(clean_file=clean_file, corrupt_file=noisy_file, data_dir=test_data_path)

../dataset/test clean_en_lyrics.txt noisy_en_lyrics.txt


930it [00:00, 464258.83it/s]
930it [00:00, 929181.21it/s]


loaded tuples of (corr,incorr) examples from ../dataset/test
###############################################
data size: 930


233it [00:56,  4.11it/s]


Epoch None valid_loss: 0.10733045486783492
total inference time for this data is: 56.645014 secs
###############################################


total token count: 6684
_corr2corr:5348, _corr2incorr:49, _incorr2corr:1193, _incorr2incorr:94
accuracy is 0.9786056253740275
word correction rate is 0.9269619269619269
precision is 0.9909208819714657
recall is 0.9827269386255053
f1 score is 0.9868069010056278
###############################################


### Result analysis

In [33]:
bert_correct_new = bert_checker_new.correct_strings(noisy_test)
bert_correct_new[:10]

['tell her that she beautiful every day i remind her',
 'i get lost in her eyes like dust from the skies',
 "did i say that out loud i ' m so crazy about mine",
 "but we don ' t have the same soul",
 'tell me that you love me baby say it again',
 "repeat it over and over until it ' s in my brain",
 "you need to send your location i can ' t think",
 'did i say that out loud',
 "i ' m so crazy about mine",
 'when i look back']

In [34]:
clean_test_prep = [i.replace(' ', '').strip().lower() for i in clean_test]
bert_correct_new_prep = [i.replace(' ', '').strip().lower() for i in bert_correct_new]

print('len data:', len(clean_test_prep))
print('correct data after bert:', sum(a == b for a, b in zip(bert_correct_new_prep, clean_test_prep)))

len data: 930
correct data after bert: 799


In [35]:
for i in range(len(clean_test_prep)):
    if clean_test_prep[i] != bert_correct_new_prep[i]:
        print(clean_test[i], ' | ', bert_correct_new[i])

tell her that she beautiful every day i remind her  |  tell her that she beautiful every day i remind her
don't you dare say  |  dont you dare say
if don't really care see  |  if don ' t really care so
don't know what they mean they're special just for you  |  dont know what they mean they ' re special just for you
hey i could take a big bat  |  hey i could take a big bet
you could put it on a swing  |  you could put it on a song
i could fuck him in the rain  |  i could fuck hmm in the rain
i could fuck him in the range  |  i could fuck him in the rage
skirt off fuck in the backseat  |  skirt off luck in the backseat
take that shirt off baby put it on me  |  take that shit off baby put it on me
but one day i woke up like  |  but one day i walked up like
there's a whole world outside  |  there ' s a whole world outdone
for and for shut up shut up  |  four and four shut up shut up
who never got to fly at night shut up shut up  |  which never got to fly at night shut up shut up
out of the

## SC-LSTM plus ELMO (at input)

### Load pretrained model

In [36]:
# create SclstmChecker
elmo_checker = SclstmChecker()
# add ELMO at input
elmo_checker = elmo_checker.add_("elmo", at="input")  # "elmo" or "bert", "input" or "output"
# load pretrained model
elmo_checker.from_pretrained()

loading vocab from path:e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise\vocab.pkl
initializing model
Number of parameters in the model: 209906438
Loading model params from checkpoint dir: e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise
new model loaded: <class 'neuspell.corrector_elmosclstm.ElmosclstmChecker'>
loading vocab from path:e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise\vocab.pkl
initializing model
Number of parameters in the model: 209906438
Loading model params from checkpoint dir: e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise


### Evaluate pretrained model on testing set

In [37]:
elmo_checker.evaluate(clean_file=clean_file, corrupt_file=noisy_file, data_dir=test_data_path)

../dataset/test clean_en_lyrics.txt noisy_en_lyrics.txt


930it [00:00, 464922.85it/s]
930it [00:00, 928738.74it/s]


loaded tuples of (corr,incorr) examples from ../dataset/test
data size: 930


233it [02:05,  1.85it/s]


Epoch None valid_loss: 0.7947396991750457
total inference time for this data is: 125.894886 secs
###############################################
total token count: 6250
corr2corr:4634, corr2incorr:329, incorr2corr:1075, incorr2incorr:212
accuracy is 0.91344
word correction rate is 0.8352758352758353
precision is 0.9337094499294781
recall is 0.9562525794469666
f1 score is 0.944846569477011
###############################################


### Example of wrong outputs

In [38]:
# correct noisy data
elmo_correct = elmo_checker.correct_strings(noisy_test)
elmo_correct[:10]

['tell her that she beautiful every day I remind her',
 'I get lost in her eyes like dust from the skies',
 "did I say that out loud I 'm so crazy about mine",
 'but we do not have the same soul',
 'tell me that you love me baby say it again',
 "repeat it over and over until it 's in my brain",
 'you need to send your location i can not think',
 'did I say that out loud',
 "I 'm so crazy about mine",
 'when I look back']

In [39]:
clean_test_prep = [i.replace(' ', '').strip().lower() for i in clean_test]
elmo_correct_prep = [i.replace(' ', '').strip().lower() for i in elmo_correct]

print('len data:', len(clean_test_prep))
print('correct data after bert:', sum(a == b for a, b in zip(elmo_correct_prep, clean_test_prep)))

len data: 930
correct data after bert: 593


In [40]:
for i in range(len(clean_test_prep)):
    if clean_test_prep[i] != elmo_correct_prep[i]:
        print(clean_test[i], ' | ', elmo_correct[i])

tell her that she beautiful every day i remind her  |  tell her that she beautiful every day I remind her
but we don't have the same soul  |  but we do not have the same soul
you need to send your location i can't think  |  you need to send your location i can not think
and if i do will you be there with me father sister brother  |  and if I do will you be there with my father sister brother
you said that i might  |  you said that it might
i guess i'll be fine  |  I guess will be fine
don't you dare say  |  do not you dare say
if you don't really care  |  if you do not really care
but lately i can't see  |  but lately I cannot see
can't you see it  |  can not you see it
don't you see it  |  do not you see it
baby don't you dare say  |  baby do not you dare baby
maybe i'll take my glasses off  |  maybe will take my glasses off
don't just say it  |  not just say it
baby if you care then don't you dare say  |  baby if you care then do not you dare say
yeah don't you dare say  |  yeah not 

### Fine-tuning SclstmChecker

In [12]:
# create AclstmChecker
elmo_checker_new = SclstmChecker()
# add elmo at input
elmo_checker_new = elmo_checker_new.add_("elmo", at="input")  # "elmo" or "bert", "input" or "output"
# load pretrained model
elmo_checker_new.from_pretrained()
# fine-tune on training data
elmo_checker_new.finetune(clean_file=clean_file, corrupt_file=noisy_file, data_dir=train_data_path)

loading vocab from path:e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise\vocab.pkl
initializing model
Number of parameters in the model: 209906438
Loading model params from checkpoint dir: e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise
new model loaded: <class 'neuspell.corrector_elmosclstm.ElmosclstmChecker'>
loading vocab from path:e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise\vocab.pkl
initializing model
Number of parameters in the model: 209906438
Loading model params from checkpoint dir: e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise


7309it [00:00, 811868.85it/s]
7309it [00:00, 1828145.26it/s]


loaded tuples of (corr,incorr) examples from ../dataset/train
len of train and test data:  5848 1461
CHECKPOINT_PATH: e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise\new_models\elmo-base-cased-1
Training model params from scratch
In epoch: 0
train_data size: 5848
Percent: [----------------------------->] 100% || batch_time: 1.8574 || batch_loss: 0.0289 || avg_batch_loss: 0.2516 || batch_acc: 0.8835 || avg_batch_acc: 0.8835 
Epoch 0 train_loss: 0.25164507261268754
valid_data size: 1461
Percent: [----------------------------->] 100% || batch_time: 1.6337 || batch_loss: 0.0423 || avg_batch_loss: 0.0786 || batch_acc: 0.9800 || avg_batch_acc: 0.9773 
Epoch 0 valid_loss: 0.07855986807819294
Model saved at e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise\new_models\elmo-base-cased-1\model.pth.tar in epoch 0
In epoch: 1
train_data size: 5848
Percent: [----------------------------->] 100% || batch_time: 2.0445 || batch_loss: 0.0225 || avg

### Evaluate on testing set after fine-tuning

In [41]:
elmo_checker_new.evaluate(clean_file=clean_file, corrupt_file=noisy_file, data_dir=test_data_path)

../dataset/test clean_en_lyrics.txt noisy_en_lyrics.txt


930it [00:00, 465200.09it/s]
930it [00:00, 930289.22it/s]


loaded tuples of (corr,incorr) examples from ../dataset/test
data size: 930


233it [02:06,  1.85it/s]


Epoch None valid_loss: 0.06675239470649322
total inference time for this data is: 126.218415 secs
###############################################
total token count: 6250
corr2corr:4939, corr2incorr:24, incorr2corr:1131, incorr2incorr:156
accuracy is 0.9712
word correction rate is 0.8787878787878788
precision is 0.9951642151924239
recall is 0.9693817468105986
f1 score is 0.9821037979717637
###############################################


### Result analysis

In [42]:
elmo_correct_new = elmo_checker_new.correct_strings(noisy_test)
elmo_correct_new[:10]

['tell her that she beautiful every day i remind her',
 'i get lost in her eyes like dust from the skies',
 "did i say that out loud i 'm so crazy about mine",
 'but we do not have the same soul',
 'tell me that you love me baby say it again',
 "repeat it over and over until it 's in my brain",
 'you need to send your location i can not think',
 'did i say that out loud',
 "i 'm so crazy about mine",
 'when i look back']

In [43]:
clean_test_prep = [i.replace(' ', '').strip().lower() for i in clean_test]
elmo_correct_new_prep = [i.replace(' ', '').strip().lower() for i in elmo_correct_new]

print('len data:', len(clean_test_prep))
print('correct data after bert:', sum(a == b for a, b in zip(elmo_correct_new_prep, clean_test_prep)))

len data: 930
correct data after bert: 720


In [44]:
for i in range(len(clean_test_prep)):
    if clean_test_prep[i] != elmo_correct_new_prep[i]:
        print(clean_test[i], ' | ', elmo_correct_new[i])

tell her that she beautiful every day i remind her  |  tell her that she beautiful every day i remind her
but we don't have the same soul  |  but we do not have the same soul
you need to send your location i can't think  |  you need to send your location i can not think
i guess i'll be fine  |  i guess il'l be fine
don't you dare say  |  do not you dare say
if you don't really care  |  if you do not really care
but lately i can't see  |  but lately i cna't see
can't you see it  |  can not you see it
don't you see it  |  do not you see it
baby don't you dare say  |  baby do not you dare say
maybe i'll take my glasses off  |  maybe iw'll take my glasses off
don't just say it  |  dn't just say it
baby if you care then don't you dare say  |  baby if you care then do not you dare say
yeah don't you dare say  |  yeah dom't you dare say
if don't really care see  |  if do't really care side
if you're not coming home to me  |  if yuo're not coming home to me
but i've got things to say  |  but i